In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
import tensorflow as tf

a = tf.constant([[1, 2], [3, 4]])                 
b = tf.add(a, 1)

print(a.numpy())

In [ ]:
from utils import bbox_utils, data_utils, drawing_utils, io_utils, train_utils, eval_utils
from models.decoder import get_decoder_model, SSDDecoder
from models.ssd_mobilenet_v2 import get_model, init_model
from models.header import HeadWrapper

batch_size = 32
evaluate = True
use_custom_images = True
custom_image_path = "A:/data/ssd/"
backbone = "mobilenet_v2"
io_utils.is_valid_backbone(backbone)
hyper_params = train_utils.get_hyper_params(backbone)
test_data, info = data_utils.get_dataset("voc/2007", "test")
total_items = data_utils.get_total_item_size(info, "test")
labels = data_utils.get_labels(info)
labels = ["bg"] + labels
hyper_params["total_labels"] = len(labels)
img_size = hyper_params["img_size"]

data_types = data_utils.get_data_types()
data_shapes = data_utils.get_data_shapes()
padding_values = data_utils.get_padding_values()

img_paths = data_utils.get_custom_imgs(custom_image_path)
total_items = len(img_paths)
test_data = tf.data.Dataset.from_generator(lambda: data_utils.custom_data_generator(
                                           img_paths, img_size, img_size), data_types, data_shapes)
test_data = test_data.padded_batch(batch_size, padded_shapes=data_shapes, padding_values=padding_values)

ssd_model = get_model(hyper_params)
ssd_model_path = io_utils.get_model_path(backbone)
ssd_model.load_weights(ssd_model_path)

print("ssd_model_path: ", ssd_model_path)
print(hyper_params)
prior_boxes = bbox_utils.generate_prior_boxes(hyper_params["feature_map_shapes"], hyper_params["aspect_ratios"])
model = get_decoder_model(ssd_model, prior_boxes, hyper_params)

step_size = train_utils.get_step_size(total_items, batch_size)
pred_bboxes, pred_labels, pred_scores = model.predict(test_data, steps=step_size, verbose=1)

drawing_utils.draw_predictions(test_data, pred_bboxes, pred_labels, pred_scores, labels, batch_size)

In [ ]:
for (index, label) in enumerate(labels):
    print(index, label)

In [ ]:
from tensorflow.keras.models import load_model, Model
import os
sample_test_dir = "A:\\python_ai_projects\\tool_tracking\\tf-ssd\python_outputs"

if not os.path.exists(sample_test_dir):
    os.makedirs(sample_test_dir)
    
def pred_funct(image_in, model_p):
    image_expanded = np.expand_dims(image_in, axis = 0)
    tf_tensor = tf.convert_to_tensor(image_expanded.astype(np.float32))
    return model_p.predict(tf_tensor)

def generate_model(model, check_index):
    input_layer_data = model.layers[0]
    output_layer_data = model.layers[check_index]
    model1 = Model(input_layer_data.input, output_layer_data.output)
    return model1

def write_csv3d(output_data, file_name_sufix):  
    path_to_saved = os.path.join(sample_test_dir, file_name_sufix +"_python.csv")
    print(output_data.shape)
    # Loop over the rows and columns of the 3D array
    with open(path_to_saved, 'w') as file:
        for l in range(output_data.shape[3]):
            for k in range(output_data.shape[2]):
                for j in range(output_data.shape[1]):
                    for i in range(output_data.shape[0]):
                        # Join the text data in each row with commas
                        value = output_data[i, j, k, l]
                        # Write the row data to the output file
                        if i == output_data.shape[1] - 1:
                            file.write("{:.4f}".format(value))
                        else:
                            file.write("{:.4f},".format(value))
                    file.write("\n")

def write_npy(output_data, file_name_sufix):
    path_to_saved = os.path.join(sample_test_dir, file_name_sufix + ".npy")
    np.save(path_to_saved, output_data)

def write_csv(output_data, file_name_sufix):  
    path_to_saved = os.path.join(sample_test_dir, file_name_sufix +"_n.csv")
    print(output_data.shape)
    with open(path_to_saved, 'w') as file:
        for k in range(output_data.shape[2]):
            for j in range(output_data.shape[0]):
                for i in range(output_data.shape[1]):
                    # Join the text data in each row with commas
                    value = output_data[j, i, k]
                    # Write the row data to the output file
                    if i == output_data.shape[1] - 1:
                        file.write("{:.4f}".format(value))
                    else:
                        file.write("{:.4f},".format(value))
                file.write("\n")
def write_csv_2d(output_data, file_name_sufix):  
    path_to_saved = os.path.join(sample_test_dir, file_name_sufix +"_n.csv")
    print(output_data.shape)
    with open(path_to_saved, 'w') as file:
        for j in range(output_data.shape[0]):
            for i in range(output_data.shape[1]):
                # Join the text data in each row with commas
                value = output_data[j, i]
                # Write the row data to the output file
                if i == output_data.shape[1] - 1:
                    file.write("{:.4f}".format(value))
                else:
                    file.write("{:.4f},".format(value))
            file.write("\n")
def write_csv_1d(output_data, file_name_sufix):  
    path_to_saved = os.path.join(sample_test_dir, file_name_sufix +"_n.csv")
    print(output_data.shape)
    with open(path_to_saved, 'w') as file:
        for k in range(output_data.shape[0]):
            value = output_data[k]
            # Write the row data to the output file
            file.write("{:.4f}\n".format(value))

def runthis(mode1, img_data, print_input, layer_name):
    final_output = pred_funct(img_data, model1).squeeze()
    if print_input:
        print("-----------------")
        for k in range(3):
            for i in range(6):
                for j in range(6):
                    print(img_data[i, j, k], end=", ")
                print()
            print("\n")
        print(img_data.shape)
    print(final_output.shape)
    print("-----------------")
    expected_shape = len(final_output.shape)
    if expected_shape == 1:
        for j in range(min(7, final_output.shape[0])):
            print(final_output[j], end=", ")
        write_csv_1d(final_output, layer_name)  
    elif expected_shape == 2:
        for j in range(min(7, final_output.shape[0])):
            for l in range(min(7, final_output.shape[1])):
                print(final_output[j,l], end=", ")
        write_csv_2d(final_output, layer_name)  
    else:
        for k in range(min(4, final_output.shape[2])): 
            for i in range(min(7, final_output.shape[0])):
                for j in range(min(7, final_output.shape[1])):
                    print(final_output[i, j, k], end=", ")
                print()
            print("\n")
        write_csv(final_output, layer_name)   
        #write_npy(final_output, layer_name)     
    print("-----------------")

In [ ]:
import os
#model.summary()
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
w = 300
h = 300
image_file_path = os.path.join("A:\\data\\ssd\\input.bmp")
selected_image = img_to_array(load_img(image_file_path).resize((w,h))).astype(np.uint8)

img_data = (selected_image / 128.0) - 1.0
print(sample_test_dir)
#numpy_in = os.path.join(sample_test_dir, "in_ssd.npy")
#np.save(numpy_in, np.transpose(img_data.copy().astype(np.float32),[1,0,2]))

In [ ]:
check_index = 1
check_list1 = ["1_conv_label_output", "2_conv_label_output", "3_conv_label_output", "4_conv_label_output", "5_conv_label_output", "6_conv_label_output"]
check_list2 = ["1_conv_boxes_output", "2_conv_boxes_output", "3_conv_boxes_output", "4_conv_boxes_output", "5_conv_boxes_output", "6_conv_boxes_output"]
check_list = check_list1 + check_list2
print_input = True
for index, layer in enumerate(model.layers):
    #print(index, layer.name)
    if layer.name in ["labels_head", "loc"]:
        print(layer.name)
        model1 = generate_model(model, index)
        runthis(model1, img_data, print_input, layer.name)
        print_input = False

In [ ]:
import numpy as np
import os
from tensorflow.keras.layers import Activation, Concatenate, Add, ZeroPadding2D, BatchNormalization, Conv2D, Input, MaxPooling2D, Dropout, concatenate, UpSampling2D, ReLU, InputLayer, Conv2DTranspose, DepthwiseConv2D

model_dir = "A:/models/ssd/"
model_name = "ssd"

epsilon = np.finfo(float).eps
lines_in_graph = []

model_data_dir = os.path.join(model_dir, model_name)
if not os.path.exists(model_data_dir):
    os.makedirs(model_data_dir)
    
for i, layer in enumerate(model.layers): 
    print(i, ":", layer.__class__.__name__, layer.name)
    items_to_write = [layer.__class__.__name__, layer.name]
    if isinstance(layer,InputLayer):
        inpu_shape = layer.input.shape
        items_to_write.append("")
        items_to_write.append(str(inpu_shape[2]))
        items_to_write.append(str(inpu_shape[1]))
        items_to_write.append(str(inpu_shape[3]))
    elif isinstance(layer.input, list):
        input_layer_names = []
        for i in range(len(layer.input)):
            inpu_layer_name = layer.input[i].name.split('/')[0]
            input_layer_names.append(inpu_layer_name)
        items_to_write.append("&".join(input_layer_names))
        if isinstance(layer,Add):
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,Concatenate): 
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,HeadWrapper):
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        else:
            print("---------------------------------")
            print("Error: Multi input layer not known")
            print("---------------------------------")            
    else:
        inpu_layer_name = layer.input.name.split('/')[0]
        items_to_write.append(inpu_layer_name)
        if isinstance(layer,Conv2D):
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            layer_activation = layer.activation
            if layer_activation:
                items_to_write.append(layer_activation.__name__)
            else:
                print("Warning: Activation layer present : ", layer_activation.__name__)                
                print("---------------------------------")
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            items_to_write.append(str(weights_shape[3]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            dilation_rates = layer.dilation_rate
            if dilation_rates[0] != dilation_rates[1] and dilation_rates[1] != 1:
                print("---------------------------------")
                print("Error: dilation_rates must be equal")
                print("---------------------------------")                
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
            items_to_write.append(str(layer.padding))            
        elif isinstance(layer,Conv2DTranspose):
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            items_to_write.append(str(weights_shape[3]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,DepthwiseConv2D):   
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,BatchNormalization):
            weights = layer.get_weights()
            gamma = weights[0]
            beta = weights[1]
            moving_mean = weights[2]
            moving_variance = weights[3]
            a = gamma / np.sqrt(moving_variance + epsilon)
            b = - moving_mean * a + beta
            path_to_save = os.path.join(model_data_dir, layer.name + '_mean.npy')
            np.save(path_to_save, b)
            path_to_save = os.path.join(model_data_dir, layer.name + '_variance.npy')
            np.save(path_to_save, a) 

            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,MaxPooling2D):
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,ZeroPadding2D):
            if(layer.padding[0][1] != layer.padding[1][1] and layer.padding[0][0] == 0 and layer.padding[1][0] != 0):
                print(layer.padding)
                print("---------------------------------")
                print("Error: This padding not supported") 
                print("---------------------------------")
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(layer.padding[0][0]))
            items_to_write.append(str(layer.padding[0][1]))
            items_to_write.append(str(layer.padding[1][0]))
            items_to_write.append(str(layer.padding[1][1]))
        elif isinstance(layer, ReLU):
            maxValue = layer.max_value
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(maxValue))
        elif isinstance(layer, Activation):
            activation_type = layer.get_config()['activation']
            maxValue = 0
            if activation_type == 'relu':
                items_to_write.append("relu")
                maxValue = 0
                if hasattr(layer,'max_value'):
                    maxValue = layer.max_value
                inpu_shape = layer.input.shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
                items_to_write.append(str(maxValue))
            elif activation_type == 'sigmoid':
                items_to_write.append("sigmoid")
                inpu_shape = layer.input.shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
            elif activation_type == 'softmax':
                items_to_write.append("softmax")
                inpu_shape = layer.input.shape
                output_shape = layer.output.shape
                print(output_shape)
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[2]))
            else:
                print("---------------------------------")
                print("Error: Only Relu activation with max value supported, provided is ", activation_type)
                print("---------------------------------") 
        else:
            print("Error: Layer is not identified ", layer.__class__.__name__, layer.name)
    lines_in_graph.append(",".join(items_to_write))
with open(f'{model_name}.txt', 'w') as f:
    # Write each line of the list to the file
    for line in lines_in_graph:            
        f.write(line + '\n')
    print("File is saved to " + f'{model_name}.txt')    

In [ ]:
pred_bboxes, pred_labels, pred_scores = pred_funct(img_data, model)
print(pred_bboxes.shape)
print(pred_labels.shape)
print(pred_scores.shape)
drawing_utils.draw_predictions(test_data, pred_bboxes, pred_labels, pred_scores, labels, batch_size)

In [ ]:
np.set_printoptions(precision=5, suppress=True)
filer_name = "extra1_1"
path_to_saved = os.path.join(model_data_dir, f'{filer_name}_mean.npy')
if os.path.exists(path_to_saved):
    data = np.load(path_to_saved)
    for i in range(min(5, data.size)):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
    path_to_saved = os.path.join(model_data_dir, f'{filer_name}_variance.npy')
    data = np.load(path_to_saved)
    for i in range(min(5, data.size)):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
else:
    path_to_saved = os.path.join(model_data_dir, f'{filer_name}_bias.npy')
    data = np.load(path_to_saved)
    for i in range(min(5, data.size)):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
    
path_to_saved = os.path.join(model_data_dir, f'{filer_name}_weights.npy')
if os.path.exists(path_to_saved):
    data = np.load(path_to_saved)
    if 'depth' in filer_name:
        for i in range(3):
            for j in range(3):
                print(data[i, j, 0, 0], end=", ")
            print()
        print()
        for i in range(3):
            for j in range(3):
                print(data[i, j, 1, 0], end=", ")
            print()
    elif data.shape[0] == data.shape[1] and data.shape[0] == 1 and len(data.shape) == 4:
        print(data.shape)
        for i in range(min(7, data.shape[3])):
            for j in range(min(7, data.shape[2])):     
                print(data[0,0,j,i], end=", ")   
            print("\n")
    else:
        for l in range(2):
            for k in range(4):
                for i in range(3):
                    for j in range(3):
                        print(data[j, i, k ,l], end=", ")
                    print()
                print("")
            print("+++++++++")
        print("\n--------------")
    print(data.shape)
    print("\n--------------")